In [ ]:
import os
import pandas as pd
import numpy as np
import bisect
from sklearn.model_selection import train_test_split, GroupShuffleSplit

DIR = '/workspace/pig'

### Load data

In [ ]:
# m_df = pd.read_pickle(os.path.join(DIR, 'data', "merged_df_20211101.pkl"))
m_df = pd.read_pickle("../tmp.pkl")

### Add bin column

In [ ]:
bins = [30., 40., 50., 60., 70., 80., 90., 100., 110.]
bins = bins + [float(i) for i in range(111, 125)] + [125., 130., 140.]
labels = [p.weight for idx, p in m_df.iterrows()]
_, bins = np.histogram(labels, range=range, bins=bins)
m_df['bins'] = [bisect.bisect_right(bins, label) for label in labels]

### split train val test

In [ ]:
for i in sorted(list(m_df['bins'].value_counts().index)):
    idxes = list(set(m_df[m_df['bins'] == i]['UID']))
    
    if len(idxes) <= 3:
        continue
    
    # split train and val_test
    if len(m_df[m_df['bins'] == i])*0.3 >= 200:
        gss = GroupShuffleSplit(n_splits=1, test_size=200 / m_df[m_df['bins'] == i].shape[0], random_state=0)
    else:
        gss = GroupShuffleSplit(n_splits=1, train_size=0.7, random_state=0)
    gss.get_n_splits()
    bin_df = m_df[m_df['bins'] == i]

    idxes = np.array(bin_df.index)
    groups =  list(bin_df['UID'])

    train_idx, val_test_idx = gss.split(idxes, idxes, groups).__next__()
    assert len(set(bin_df.iloc[train_idx]['UID']) & set(bin_df.iloc[val_test_idx]['UID'])) == 0

    # split val and test
    gss = GroupShuffleSplit(n_splits=1, train_size=0.5, random_state=0)
    gss.get_n_splits()
    groups =  list(bin_df.iloc[val_test_idx]['UID'])

    _val_idx, _test_idx = gss.split(val_test_idx, val_test_idx, groups).__next__()
    val_idx = val_test_idx[_val_idx]
    test_idx = val_test_idx[_test_idx]

    assert len(set(bin_df.iloc[train_idx]['UID']) & set(bin_df.iloc[val_idx]['UID']) & set(bin_df.iloc[test_idx]['UID'])) == 0
    
    print(bins[i])
    print(len(set(bin_df.iloc[train_idx]['UID'])))
    print(len(set(bin_df.iloc[val_idx]['UID'])))
    print(len(set(bin_df.iloc[test_idx]['UID'])))
    print()

    m_df.loc[idxes[train_idx], 'subset'] = 'train'
    m_df.loc[idxes[val_idx], 'subset'] = 'val'
    m_df.loc[idxes[test_idx], 'subset'] = 'test'

### Check data

In [ ]:
count, bins = np.histogram(
    [p.weight for idx, p in m_df[m_df['subset'] == 'train'].iterrows()], bins=bins)
print(m_df[m_df['subset'] == 'train'].shape)
print("bins: ", bins)
print("count: ", count)
print()

count, bins = np.histogram(
    [p.weight for idx, p in m_df[m_df['subset'] == 'val'].iterrows()], bins=bins)
print(m_df[m_df['subset'] == 'val'].shape)
print("bins: ", bins)
print("count: ", count)
print()

count, bins = np.histogram(
    [p.weight for idx, p in m_df[m_df['subset'] == 'test'].iterrows()], bins=bins)
print(m_df[m_df['subset'] == 'test'].shape)
print("bins: ", bins)
print("count: ", count)

### Save data

In [ ]:
m_df.to_pickle(os.path.join(DIR, 'data/akiyama_20211210.pkl'))

In [ ]:
# pd.read_pickle(os.path.join(DIR, 'data/akiyama_20211202.pkl'))